### Classifying students's answers

#### Import libraries

In [429]:
# Input
# from db import PythonProblems
import io

#DB
from questions.models import Solution, Cluster, UserLog
import psycopg2

# Helpers
import numpy as np
import pickle
import base64
import pandas as pd

# Preprocessing
import tokenize
from sklearn.feature_extraction.text import CountVectorizer

# Distance
from sklearn.metrics.pairwise import euclidean_distances

# Learning
from clustering import Clustering
# from questions.sampling import get_next_sample
from analyzer import python_analyzer

# Visualization
# from plot2d import Plot2D
# import matplotlib.pyplot as plt
# import seaborn as sns
# from generate_pdf import PDFReport
# import pyLDAvis.sklearn
from IPython.display import display
import ipywidgets as widgets


%matplotlib inline

In [3]:
## Cleaning database
last_id = 132
problems = Problem.objects.filter(id__gt=last_id)
solutions_obj = Solution.objects.filter(problem__in=problems).update(ignore=True)
print("Problems to be ignored: %d" % problems.count())

problems = Problem.objects.filter(id__lte=last_id)
# problems = Problem.objects.all()
print("Problems to be used: %d" % problems.count())

solutions_obj = Solution.objects.filter(problem__in=problems, ignore=False).order_by('id')
# solutions_obj = Solution.objects.all().order_by('id')
print("Solutions to be used: %d" % solutions_obj.count())

docs_id = []
questions = []
solutions = []

# Fill separated structures
for sol in solutions_obj:
    docs_id.append(sol.id)
    questions.append(sol.problem.content)
    solutions.append(sol.content)

print("Got %d documents" %(solutions_obj.count()))

Problems to be ignored: 591
Problems to be used: 132
Solutions to be used: 54
Got 54 documents


***Limitações do parser:***

***1. Variável e nome reservado são do mesmo tipo. Não consigo separar sem fazer uma pré-lista. Mas pode ser o critério para ajustar o CountVectorizer***

***2. Não é possível processar solução sintaticamente errada. Talvez uma tokenização básica possa ajudar quando esse for o caso e já é uma indicação de que está errado.***

### Preprocessing solutions into bag of words ###

In [4]:
def create_bag_of_words(docs, vectorizer_method, binary=False, min_df=0.2):
    vectorizer = vectorizer_method(analyzer = python_analyzer,
                                   binary=binary,
                                   min_df=min_df) 
    train_data_features = vectorizer.fit_transform(docs)
    try:
        train_data_features = train_data_features.toarray()
    # It's already an array
    except AttributeError:
        pass
    return train_data_features, vectorizer, vectorizer.get_feature_names()

In [5]:
# %load vectorizer.py
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
from analyzer import python_analyzer
import numpy as np


class NCutVectorizer(object):
    def __init__(self, analyzer, binary, min_df):
        self.vectorizer = CountVectorizer(analyzer = python_analyzer,
                                          binary=binary,
                                          min_df=min_df)

    def fit_transform(self, docs):
        train_data_features = self.vectorizer.fit_transform(docs)
        train_data_features = train_data_features.toarray()

        # Calculate NCut-weight
        doc_mat_norm = normalize(train_data_features)
        S = np.dot(doc_mat_norm.T, doc_mat_norm) + 0.001
        D = np.power(np.sum(S, axis=1), -0.5) * np.eye(S.shape[0])
        Y = np.dot(D, train_data_features.T)
        return Y.T
    
    def get_feature_names(self):
        return self.vectorizer.get_feature_names()

    def transform(object):
        train_data_features = self.vectorizer.transform(docs)
        train_data_features = train_data_features.toarray()
        

# Connect to DB

In [6]:
connection = psycopg2.connect(user = "machineteaching",
                                  password = "***REMOVED***",
                                  host = "localhost",
#                                   port = "5432",
                                  database = "machineteaching")
connection.autocommit=True
cursor = connection.cursor()

In [158]:
def get_where_items(exp_id):
    cols = ["vectorizer", "min_df", "is_binary", "distance", "method", "dataset", "k", "model", "X"]
    query = "SELECT %s from experiments_solution where experiment_id = %s" % (", ".join(cols), exp_id) 
    cursor.execute(query)
    where_items = cursor.fetchall()
    return where_items

def analyze(solutions, where_items, exp_id):
    v = eval(where_items[0][0])
    m = where_items[0][1]
    b = where_items[0][2]
    dist = where_items[0][3]
    method = where_items[0][4]
    k = where_items[0][6]
    model_db = pickle.loads(base64.b64decode(where_items[0][7]))
    X = np.asarray(where_items[0][8])

    train_data_features, vectorizer, feature_names = create_bag_of_words(solutions, v, binary=b, min_df=m)
    clustering = Clustering(train_data_features, k, metric=dist)
    clustering.seed = model_db.random_state
    
    model, document_topic, word_topic = getattr(clustering, method)()
    
#     savefig='problems_20190101_%s_%s_exp_%s_%s_document_topic_distribution' % (method, clustering.seed, 
#                                                                                    exp_id, clustering.k)
#     clustering.plot_topic_distribution(title="Document distribution per topic", ylabel="Document ID", savefig=savefig)

#     print("Count per class:")
#     clusters = clustering.document_topic.argmax(axis=1)
#     counts = np.unique(clusters, return_counts=True)
#     print(counts)
    return model, vectorizer

def transform(student, solutions, model_db, vectorizer, exp_id):
    train_data_features_student = vectorizer.transform([student]).toarray()
    train_data_features_solutions = vectorizer.transform(solutions).toarray()
    
    X = np.concatenate((train_data_features_student, train_data_features_solutions), axis=0)

    y_student = model_db.transform(train_data_features_student)
    y_solutions = model_db.transform(train_data_features_solutions)
    return X, y_student, y_solutions
#     print("Count per class:")
#     clusters = clustering.document_topic.argmax(axis=1)
#     counts = np.unique(clusters, return_counts=True)
#     print(counts)
#     return clustering, method, feature_names, model

In [8]:
import random

In [132]:
def get_student_exercises():
    student = random.choice(User.objects.all())
    exercises = UserLog.objects.filter(user=student, outcome__in=['P', 'F']).order_by('timestamp')
    return student, exercises

In [129]:
exercise = exercises[1]
student_solution = exercise.solution
print(student_solution)
solutions_problem = Solution.objects.filter(problem=exercise.problem, ignore=False).values_list('content', flat=True)

for item in solutions_problem:
    print(item)

# Complete your function using this header
def amount(transaction_list):
    soma = 0 
    for i in range(len(transaction_list)):
        if transaction_list[i][0] == 'D':
            soma = soma + transaction_list[i][1]
        else:
            soma = soma - transaction_list[i][1]
    return soma
# Complete your function using this header
def amount(transaction_list):
    op = {"D": 1, "W": -1}
    values = [op[item[0]]*item[1] for item in transaction_list]
    return sum(values)
def amount(transaction_list):
    total = 0
    for item in transaction_list:
        op, value = item
        if op == 'D':
            total += value
        elif op == 'W':
            total -= value
    return total



# Experiment 1463

- Min DF: 0.35
- Binary: True
- Vectorizer: NCut
- Method: NMF
- Best k: 7

In [66]:
# Get experiment conditions
exp_id = 1463
where_items = get_where_items(exp_id)
print("Conditions")
print(where_items[0][0:7])

model_db = pickle.loads(base64.b64decode(where_items[0][7]))
model, vectorizer  = analyze(solutions, where_items, exp_id)
y_student, y_solutions = transform(student_solution, solutions, model_db, vectorizer, exp_id)



Conditions
('NCutVectorizer', 0.35, True, 'euclidean', 'nmf', 'solution_all', 7)


/home/lmoraes/miniconda3/envs/machine_teaching/lib/python3.6/site-packages/sklearn/base.py:311: UserWarning: Trying to unpickle estimator NMF from version 0.20.1 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


AttributeError: 'NCutVectorizer' object has no attribute 'transform'

# Experiment 26

- Min DF: 0.05
- Binary: True
- Vectorizer: Count
- Method: LDA
- Best k: 12

In [362]:
def get_proportions(student_solution, possible_solutions):
    # Get experiment conditions
    exp_id = 26
    where_items = get_where_items(exp_id)
#     print("Conditions")
#     print(where_items[0][0:7])

    model_db = pickle.loads(base64.b64decode(where_items[0][7]))
    model_db.n_jobs = 1
    model, vectorizer = analyze(solutions, where_items, exp_id)

    X, y_student, y_solutions = transform(student_solution, possible_solutions, model_db, vectorizer, exp_id)

    c = Clustering(X, where_items[0][6])
    c.document_topic = np.concatenate((y_student, y_solutions), axis=0)
    # c.plot_topic_distribution()
#     print(np.array([xi * 100//1 for xi in c.document_topic]))
    return c.document_topic

#     X, y_student, y_solutions = transform(student_solution, solutions_problem, model, vectorizer, exp_id)
#     c = Clustering(X, where_items[0][6])
#     c.document_topic = np.concatenate((y_student, y_solutions), axis=0)
#     print(np.array([xi * 100//1 for xi in c.document_topic]))
    # c.plot_topic_distribution()

In [134]:
# Constants
# Initial learning state
L0 = 0
# Probability of transition
T = 0.1
# Probability of slip
S = 0.3
# Probability of guess
G = 0.3

In [392]:
def correct(learning_state):
    learning_correct = learning_state * (1 - S)
    guess_correct = ((1 - learning_state) * G)
    learning_evidence = learning_correct/(learning_correct + guess_correct)
    return learning_evidence

def incorrect(learning_state):
    learning_incorrect = learning_state * S
    guess_incorrect = ((1 - learning_state) * (1 - G))
    learning_evidence = learning_incorrect/(learning_incorrect + guess_incorrect)
    return learning_evidence
    
def update(learning_state, distribution, iscorrect=True):
    if iscorrect:
        learning_evidence = correct(learning_state)
    else:
        learning_evidence = incorrect(learning_state)
    print("Learning evidence: %s" % learning_evidence)
    learning_state = learning_evidence + ((1 - learning_evidence) * T * distribution)
    return learning_state

In [377]:
l = 0

In [378]:
l = update(l, True)
print(l)

Learning evidence: 0.0
0.1


In [569]:
def get_new_student(b):
#     global student
#     global exercises
#     student, exercises = get_student_exercises()
    print("Student %s submitted %d responses" % (student.email, exercises.count()))
    
def update_learning(b):
    try:
        idx, distribution, outcome = next(l)
    except StopIteration:
        label_finished.value = "Finished"
    
    label_submission.value = "Submission " + str(idx+1)
    if outcome == "Passed":
        layout=widgets.Layout(border='1px solid green')
    else:
        layout=widgets.Layout(border='1px solid red')
    label_outcome.value = "Outcome: " + outcome
    label_outcome.layout = layout
    
    # Topic 6: simple math
    simple_math.value = distribution[5]*100//1
    label_simple_math.value = str(simple_math.value)
    # Topic 8: conditional
    conditional.value = distribution[7]*100//1
    label_conditional.value = str(conditional.value)
    # Topic 12: loop 
    loop.value = distribution[11]*100//1
    label_loop.value = str(loop.value)
    # Topic 10: list manipulation
    list_manipulation.value = distribution[9]*100//1
    label_list_manipulation.value = str(list_manipulation.value)
    # Topics 2 and 4: string
    string.value = (distribution[1] + distribution[3])*100//1
    label_string.value = str(string.value)
    
def go_to_next_passed(b):
    outcome = "Failed"
    
    try:
        while outcome != "Passed":
            idx, distribution, outcome = next(l)
    except StopIteration:
        label_finished.value = "Finished"

    label_submission.value = "Submission " + str(idx+1)
    if outcome == "Passed":
        layout=widgets.Layout(border='1px solid green')
    else:
        layout=widgets.Layout(border='1px solid red')
    label_outcome.value = "Outcome: " + outcome
    label_outcome.layout = layout
    
    # Topic 6: simple math
    simple_math.value = distribution[5]*100//1
    label_simple_math.value = str(simple_math.value)
    # Topic 8: conditional
    conditional.value = distribution[7]*100//1
    label_conditional.value = str(conditional.value)
    # Topic 12: loop 
    loop.value = distribution[11]*100//1
    label_loop.value = str(loop.value)
    # Topic 10: list manipulation
    list_manipulation.value = distribution[9]*100//1
    label_list_manipulation.value = str(list_manipulation.value)
    # Topics 2 and 4: string
    string.value = (distribution[1] + distribution[3])*100//1
    label_string.value = str(string.value)
    
def calculate_learning_state():
    l = np.zeros((12,))
    for idx, exercise in enumerate(exercises):
        student_solution = exercise.solution
        solutions_problem = Solution.objects.filter(problem=exercise.problem, ignore=False).values_list('content', flat=True)
        solutions_students = UserLog.objects.filter(problem=exercise.problem, outcome='P').values_list('solution', flat=True)

        
        possible_solutions = list(solutions_problem) + list(solutions_students)
        # Hyp 1: use the closest correct answer
        if hyp == 1:
            document_topic = get_proportions(student_solution, possible_solutions)
        # Hyp 2: use only original answers
        elif hyp == 2:
            document_topic = get_proportions(student_solution, solutions_problem)
        # Hyp 3: use only student answer
        elif hyp == 3:
            document_topic = get_proportions(student_solution, [student_solution])
        
        idx_model_answer = np.argmin(euclidean_distances(document_topic, document_topic)[0,1:])+1
        print(np.array([xi * 100//1 for xi in document_topic[[0,idx_model_answer],:]]))

        print("Submission #%d: %s" %(idx+1, exercise.outcome))
        distribution = document_topic[idx_model_answer,:]
        if exercise.outcome == 'P':
            outcome = "Passed"
            l = update(l, distribution, True)
            print(exercise.solution)
            print(possible_solutions[idx_model_answer-1])
        else:
            outcome = "Failed"
            l = update(l, distribution, False)
            print(exercise.solution)
            print(possible_solutions[idx_model_answer-1])
        print("Learning state: %s" % (l*100//1))
        yield idx, l, outcome

In [518]:
new_student_button = widgets.Button(description="Get new student")
display(new_student_button)
new_student_button.on_click(get_new_student)

Button(description='Get new student', style=ButtonStyle())

Student livia2000medeiros@gmail.com submitted 20 responses
Student livia2000medeiros@gmail.com submitted 20 responses


In [570]:
button = widgets.Button(description="Update learning state")
display(button)
button.on_click(update_learning)

button_next_p = widgets.Button(description="Next passed state")
display(button_next_p)
button_next_p.on_click(go_to_next_passed)

label_finished = widgets.Label(value='')
display(label_finished)

# Topics/Concepts
simple_math = FloatProgress(min=0, max=100,  bar_style='info', description="Simple Math")
conditional = FloatProgress(min=0, max=100,  bar_style='warning', description="Conditional")
loop = FloatProgress(min=0, max=100,  bar_style='danger', description="Loop")
list_manipulation = FloatProgress(min=0, max=100,  bar_style='success', description="List Manipulation")
string = FloatProgress(min=0, max=100,  bar_style='', description="String")

# Labels
label_submission = widgets.Label(value="Submission "+str(0))
style = {'border': 'solid', "border-color": "black"}
label_outcome = widgets.Label(value="Outcome:")
label_simple_math = widgets.Label(value=str(0))
label_conditional = widgets.Label(value=str(0))
label_loop = widgets.Label(value=str(0))
label_list_manipulation = widgets.Label(value=str(0))
label_string = widgets.Label(value=str(0))

display(label_submission)
display(label_outcome)
display(widgets.HBox([simple_math, label_simple_math]))
display(widgets.HBox([conditional, label_conditional]))
display(widgets.HBox([loop, label_loop]))
display(widgets.HBox([list_manipulation, label_list_manipulation]))
display(widgets.HBox([string, label_string]))

hyp = 3
l = calculate_learning_state()

Button(description='Update learning state', style=ButtonStyle())

Button(description='Next passed state', style=ButtonStyle())

Label(value='')

Label(value='Submission 0')

Label(value='Outcome:')

...

/home/lmoraes/miniconda3/envs/machineteaching/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/lmoraes/miniconda3/envs/machineteaching/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[[ 0.  0.  0.  0.  0. 19.  0. 77.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 19.  0. 77.  0.  0.  0.  0.]]
Submission #19: F
Learning evidence: [0.00045277 0.00075975 0.00035747 0.00035747 0.00035747 0.03644155
 0.00123691 0.00750792 0.00035747 0.00044619 0.00035747 0.0351612 ]
# Complete your function using this header
def divisible(n):
    if n/7==0:
        return n
def divisible():
    l=[]
    for i in range(2000, 3201):
        if (i%7==0) and (i%5!=0):
            l.append(i)
    return l


Learning state: [0. 0. 0. 0. 0. 5. 0. 8. 0. 0. 0. 3.]


/home/lmoraes/miniconda3/envs/machineteaching/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[[ 0.  0.  0.  0.  0. 27. 11. 57.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 27. 11. 57.  0.  0.  0.  0.]]
Submission #20: F
Learning evidence: [0.00034295 0.00047461 0.00030208 0.00030208 0.00030208 0.02439637
 0.00067935 0.03784338 0.00030208 0.00034012 0.00030208 0.01552762]
# Complete your function using this header
def divisible():
    for num/7==0:
        return n
def divisible():
    l=[]
    for i in range(2000, 3201):
        if (i%7==0) and (i%5!=0):
            l.append(i)
    return l


Learning state: [0. 0. 0. 0. 0. 5. 1. 9. 0. 0. 0. 1.]


UnboundLocalError: local variable 'idx' referenced before assignment

In [393]:
for idx, exercise in enumerate(exercises):
    student_solution = exercise.solution
    solutions_problem = Solution.objects.filter(problem=exercise.problem, ignore=False).values_list('content', flat=True)
    solutions_students = UserLog.objects.filter(problem=exercise.problem, outcome='P').values_list('solution', flat=True)
    
    # Hyp 1: use the closest correct answer
    possible_solutions = list(solutions_problem) + list(solutions_students)
    document_topic = get_proportions(student_solution, possible_solutions)
    idx_model_answer = np.argmin(euclidean_distances(document_topic, document_topic)[0,1:])+1
    print(np.array([xi * 100//1 for xi in document_topic[[0,idx_model_answer],:]]))
    
    print("Submission #%d: %s" %(idx+1, exercise.outcome))
    distribution = document_topic[idx_model_answer,:]
    if exercise.outcome == 'P':
        l = update(l, distribution, True)
        print(exercise.solution)
        print(possible_solutions[idx_model_answer-1])
    else:
        l = update(l, distribution, False)
    print("Learning state: %s" % (l*100//1))

Student livia2000medeiros@gmail.com submitted 20 responses


/home/lmoraes/miniconda3/envs/machineteaching/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[[ 0.  0.  0.  0.  0. 86.  0.  0.  0.  0.  0.  8.]
 [ 0.  0.  0.  0.  0. 86.  0.  0.  0.  0.  0.  8.]]
Submission #1: F
Learning evidence: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Learning state: [0. 0. 0. 0. 0. 8. 0. 0. 0. 0. 0. 0.]


/home/lmoraes/miniconda3/envs/machineteaching/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[[ 0.  0.  0.  0.  0. 86.  0.  0.  0.  0.  0.  8.]
 [ 0.  0.  0.  0.  0. 86.  0.  0.  0.  0.  0.  8.]]
Submission #2: F
Learning evidence: [0.0002233  0.00022328 0.00022329 0.00022328 0.00022328 0.03879541
 0.00022328 0.00022328 0.00022328 0.00022328 0.00022328 0.00375635]
Learning state: [ 0.  0.  0.  0.  0. 12.  0.  0.  0.  0.  0.  1.]


...

[[ 0.  0.  0.  0.  0. 44.  0.  0.  0.  0.  0. 50.]
 [ 0.  0.  0.  0.  0.  0.  0. 33.  0.  0.  0. 64.]]
Submission #18: F
Learning evidence: [0.00155944 0.00113202 0.00019596 0.00019597 0.00019596 0.01143851
 0.00019596 0.03380916 0.00019596 0.00204487 0.00019596 0.03950216]
Learning state: [ 0.  0.  0.  0.  0.  1.  0.  6.  0.  0.  0. 10.]


/home/lmoraes/miniconda3/envs/machineteaching/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[[ 0.  0.  0.  0.  0. 19.  0. 77.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 23.  0. 51.  0.  0.  0. 22.]]
Submission #19: F
Learning evidence: [0.00075399 0.00057052 0.00016904 0.00016904 0.00016904 0.00501966
 0.00016904 0.02938622 0.00016904 0.00096247 0.00016904 0.04625675]
Learning state: [0. 0. 0. 0. 0. 2. 0. 7. 0. 0. 0. 6.]


/home/lmoraes/miniconda3/envs/machineteaching/lib/python3.7/site-packages/sklearn/base.py:253: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.20.1 when using version 0.20.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


[[ 0.  0.  0.  0.  0. 27. 11. 57.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0. 23.  0. 51.  0.  0.  0. 22.]]
Submission #20: F
Learning evidence: [0.00042252 0.00034382 0.00017168 0.00017168 0.00017168 0.01251352
 0.00017168 0.03549789 0.00017167 0.00051195 0.00017167 0.03036359]
Learning state: [0. 0. 0. 0. 0. 3. 0. 8. 0. 0. 0. 5.]
